In [146]:
#py_pandas_pivot_tables.ipynb
import numpy as np              
import pandas as  pd            
import matplotlib.pyplot as plt 
%matplotlib inline
import os
datafile='earthquakes.csv'

#useful if you want the directory this script is in
if '__file__' in dir():
    path, _=os.path.split(__file__)  
else: path=os.getcwd() 
    
filename=os.path.join(path, datafile)
df=pd.read_csv(filename, parse_dates=[0]) #dialect defaults to Excel

In [147]:
#py_pandas_pivot_tables.ipynb

#use time for the index
df['place']=df['place'].astype(str)
df['place']=df['place'].str.split(',').str.get(1)
df['place']=df['place'].str.strip()
df['place']=df['place'].str.replace("CA", "California")

#  make year, month, and day columns
df['year']=df['time'].dt.year
df['mon']=df['time'].dt.month
df['day']=df['time'].dt.day

#nuke extraneous columns
df.drop(df.columns[[1,2,5,6,7,8,9,11,12,14,15,16,17,18,20,21]], \
        axis=1, inplace=True)

#make time the index
df.set_index('time', inplace=True)

df.head(3)

,depth,mag,net,place,status,year,mon,day
time,,,,,,,,
2017-09-19 16:56:50.540,1.52,2.45,ci,California,automatic,2017,9,19
2017-09-19 16:15:29.900,2.63,2.92,nc,California,automatic,2017,9,19
2017-09-19 15:35:09.030,4.52,3.10,us,Idaho,reviewed,2017,9,19


In [ ]:
#create a pivot table
pd.set_option("display.width", 100)
pt=pd.pivot_table(df, index=['place', 'year', 'mon'])
pt[:5]

In [ ]:
#to rearrange the groupings, simply reorder the indices
pt=pd.pivot_table(df, index=['mon', 'place', 'year'])
pt[:5]

In [ ]:
pt=pd.pivot_table(df, index=['place'],values=['mag'], 
                  aggfunc=[np.mean, len])
pt[:5]

In [ ]:
pt=pd.pivot_table(df, index=['place', 'year', 'mon', 'day', 'depth'])
pt.query("place == ['Oregon', 'California'] and mag > 4")

In [ ]:
type(pt)

In [ ]:
cut_offs=[0,4,6,8,10]
names=['weak', 'moderate', 'strong', 'intense']
df['strength']=pd.cut(df['mag'], cut_offs, labels=names)
pt=pd.pivot_table(df, index=['place', 'year', 'mon', 'day', 'depth', 'strength'])
pt.query("place == ['Oregon', 'California'] and mag > 4")

In [ ]:
def classify(data_row):
    depth=data_row['depth']
    if depth < 10:     return 'surface'
    elif depth < 100:  return 'shallow'
    elif depth < 1000: return 'kinda deep'
    else:              return 'really deep'
df['deep_cat']=df.apply(classify, axis=1)
df[:5]

In [ ]:
pd.crosstab(df.strength, columns= df.deep_cat, margins=True )

In [ ]:
pd.crosstab([df.mon, df.strength], columns= df.deep_cat, margins=True )

In [ ]:
pd.crosstab([df.mon, df.strength], columns= [df.deep_cat, df.year], margins=True )